# Text context

In [9]:
import pymongo

# Create the connection to MongoDB
try:
    connection=pymongo.MongoClient()
    print "Connection to Mongo Daemon successful!!!"
except pymongo.errors.ConnectionFailure, e:
    print "Could not connect to MongoDB: %s" % e
    # Obtenim la BD del Congrés
db = connection['congres']
print "Collections : ", db.collection_names()

Connection to Mongo Daemon successful!!!
Collections :  [u'system.indexes', u'legislatures', u'all_document', u'document', u'congres_members', u'congres_groups']


## Get documents from mongodb

In [15]:
import datetime
import re
# Col.lecció de documents a la BD
p = re.compile(r'\b[a-zA-Z\xe1\xe9\xed\xf3\xfa\xc1\xc9\xcd\xd3\xda\xf1\xe7]+\b',re.UNICODE)

doc_col = db['document']

db_has_keywords = False

doc_start_date = "08/02/2012"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
doc_end_date = "01/04/2015"
d_doc_end_date = datetime.datetime.strptime(doc_end_date, "%d/%m/%Y")

result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)

documents = []
text_documents = [] # documentos = [d1=p1+i11+...+i1M, ..., dN=pN+iN1+...]
docs_tokenized_keywords = []
    
if result.count() > 0 :
    for doc in result:
        print doc['description']
        dialog_tokenized_keywords = []
        for dialogo in doc['session_dictionary']:
            q_text = dialogo['question']
            if db_has_keywords:
                q_tokenized_keywords = []
                for keyword in dialogo['keywords']:
                    q_tokenized_keywords += keyword[0].split()
                i_tokenized_keywords = []
            i_text = ""
            for intervencion in dialogo['intervention_dictionary']:
                if db_has_keywords:
                    for keyword in intervencion['keywords']:
                        i_tokenized_keywords += keyword[0].split()
                i_text += (" " + intervencion['text'])
            d_text =  q_text + " " + i_text
            if db_has_keywords:
                dialog_tokenized_keywords += (q_tokenized_keywords + i_tokenized_keywords)
                documents.append({'date':doc['date'], 'question':q_text, 'interventions':i_text, 'keywords':(q_tokenized_keywords + i_tokenized_keywords)})
            text_documents.append(d_text)
        if db_has_keywords:
            docs_tokenized_keywords += dialog_tokenized_keywords

if db_has_keywords:
    print "Num. of tokenized keywords found in sessions:", len(docs_tokenized_keywords)
    set_docs_tokenized_keywords = set(docs_tokenized_keywords)
    print "Num. of tokenized keywords:", len(set_docs_tokenized_keywords)
print "Num. text documents:", len(text_documents)

Congreso de los Diputados, Pleno y Dip. Perm., núm. 7, de 08/02/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 10, de 15/02/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 13, de 22/02/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 16, de 07/03/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 19, de 14/03/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 22, de 28/03/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 25, de 11/04/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 30, de 16/05/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 36, de 30/05/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 42, de 20/06/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 45, de 27/06/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 50, de 18/07/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 56, de 12/09/2012
Congreso de los Diputados, Pleno y Dip. Perm., núm. 59, de 19/09/2012
Congreso de los Diput

In [13]:
# num documents
if db_has_keywords:
    print len(documents)
print len(text_documents)

1221


In [6]:
from nltk.text import Text
from nltk.corpus import brown
text = Text(brown.words(categories='news'))
print brown.words(categories='news')
print(text)

[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]
<Text: The Fulton County Grand Jury said Friday an...>


In [17]:
import nltk

tokenized_text_documents = []
for text_document in text_documents:
    # print "-" * 120
    # print text_document[:120]
    tokenized_text_document = nltk.word_tokenize(text_document)
    # print tokenized_text_document[:10]
    tokenized_text_documents.append(tokenized_text_document)

print text_documents[0][:200]
print tokenized_text_documents[0][:20]

QUÉ REFORMA LABORAL ESTÁ PREPARANDO EL GOBIERNO QUE USTED
    MISMO CREE QUE MERECE UNA HUELGA GENERAL   Señor presidente del Gobierno ¿qué reforma
    laboral piensa presentar a este Parlamento que u
[u'QU\xc9', u'REFORMA', u'LABORAL', u'EST\xc1', u'PREPARANDO', u'EL', u'GOBIERNO', u'QUE', u'USTED', u'MISMO', u'CREE', u'QUE', u'MERECE', u'UNA', u'HUELGA', u'GENERAL', u'Se\xf1or', u'presidente', u'del', u'Gobierno']


In [19]:
# http://www.nltk.org/_modules/nltk/text.html
from nltk.text import ContextIndex

def my_function_context(tokens, i, order = 1):
    """One left token and one right token, normalized to lowercase"""
    left = (tokens[i-order].lower() if i != 0 else '*START*')
    right = (tokens[i+order].lower() if i != len(tokens) - 1 else '*END*')
    return (left, right)

my_content_index = ContextIndex(tokenized_text_documents[0], context_func = my_function_context)
print my_content_index.word_similarity_dict("HUELGA")

{u'explicado': 0, u'5': 0, u'ah\xed': 0, u'pido': 0, u'much\xedsimos': 0, u'tanto': 0, u'despido': 0, u'dije': 0, u'estoy': 0, u'precedentes': 0, u'verdad': 0, u'objetivos': 0, u'apruebe': 0, u'Centella': 0, u'hace': 0, u'haremos': 0, u'estos': 0, u'debate': 0, u'aprobar': 0, u'resulta': 0, u'26': 0, u'tambi\xe9n': 0, u'quieren': 0, u'han': 0, u'la': 0, u'lo': 0, u'gente': 0, u'presentado': 0, u'HUELGA': 1.0, u',': 0, u'GENERAL': 0, u'temporales': 0, u'hay': 0, u'nuestros': 0, u'decir': 0, u'conoce': 0, u'personas': 0, u'sus': 0, u'he': 0, u'cree': 0, u'ponga': 0, u'dif\xedciles': 0, u'desalojadas': 0, u'...': 0, u'viviendas': 0, u'reforma': 0, u'yo': 0, u'ya': 0, u'por': 0, u'de': 0, u'creaci\xf3n': 0, u'algunos': 0, u'miembros': 0, u'j\xf3venes': 0, u'qu\xe9': 0, u'traer': 0, u'Parlamento': 0, u'USTED': 0, u'nos': 0, u'aqu\xed': 0, u'y': 0, u'afrontar': 0, u'nuestra': 0, u'rumores': 0, u'van': 0, u'se\xf1or\xedas': 0, u'huelga': 1.0, u'regalados': 0, u'ocasiones': 0, u'cosas': 0, u'm

In [41]:
# Mètode per eliminar les paraules que pertanyen a un conjunt de paraules donat (custom_set_of_words) 
def remove_custom_set_of_words(tokenized_doc, custom_set_of_words):
    return [ word for word in tokenized_doc if word not in custom_set_of_words ]

# Get spanish stopwords

# Get a stoplist from a pickle file
import pickle
# p = '/Users/jccan/Dropbox/projects/SpanishPoliticsDebate/data/pickle/stopwords_list.pickle'
p = 'C:\\Users\\pablo_000\\Dropbox\\SpanishPoliticsDebate\\data\\pickle\\stopwords_list.pickle'
f = open(p,'rb')
stopwords = pickle.load(f)
stopwords = [unicode(w,encoding='utf-8') for w in stopwords]
stpw = [u'señor', u'señores', u'señora', u'señoras', u'gobierno', u'presidente', u'presidenta',
        u'gracias', u'aplausos', u'ministro',
        u'ministra', u'ministros', u'programa', u'num_exp', u'vicepresidenta', u'partido', 
        u'diputado', u'diputada', u'euro', u'euros', u'ministerio',u'usted',u'ustedes',u'señoría', u'señorías', 
        u'parlamentario', u'parlamentarios', u'grupo', u'grupos',
        u'heredia', u'hernando',u'pérez', u'rubalcaba',u'saura', u'giménez',
        u'socialista', u'popular',u'año',u'años', u'cámara',u'montón',u'millón', u'millones',u'número',u'números']
stopwords.extend(stpw)

# Clustering text documents using k-means 

In [31]:
vectorizer = TfidfVectorizer(stop_words = stopwords, max_df=0.7, min_df=3,ngram_range=(1,1)) 
X = vectorizer.fit_transform(text_documents)

In [19]:
# from __future__ import print_function
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from time import time
import numpy as np

In [20]:
true_k = 20
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1, verbose=True)

In [21]:
print "Clustering sparse data with %s" % km
t0 = time()
km.fit(X)
print "done in %0.3fs" % (time() - t0)


Clustering sparse data with KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=20, n_init=1,
    n_jobs=1, precompute_distances=True, random_state=None, tol=0.0001,
    verbose=True)
Initialization complete
Iteration  0, inertia 2126.065
Iteration  1, inertia 1130.955
Iteration  2, inertia 1124.894
Iteration  3, inertia 1121.363
Iteration  4, inertia 1119.258
Iteration  5, inertia 1118.052
Iteration  6, inertia 1117.542
Iteration  7, inertia 1117.325
Iteration  8, inertia 1117.189
Iteration  9, inertia 1117.113
Iteration 10, inertia 1117.071
Iteration 11, inertia 1117.039
Iteration 12, inertia 1116.997
Iteration 13, inertia 1116.982
Converged at iteration 13
done in 0.249s


In [22]:
print "Top 10 terms per cluster:"
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print "Cluster %d:" % i, ' '.join(' %s' % terms[ind] for ind in order_centroids[i, :10])


Top 10 terms per cluster:
Cluster 0:  protestas  actitud  silencio  rumores  caso  actuaciones  relación  indulto  convenio  espero
Cluster 1:  sector  política  empresas  millones  industria  medidas  industrial  actividad  españa  sistema
Cluster 2:  tribunal  ley  derechos  constitucional  derecho  justicia  partidos  penal  sentencia  supremo
Cluster 3:  sanidad  autónomas  comunidades  servicios  sanitaria  salud  sociales  medicamentos  millones  ciudadanos
Cluster 4:  reforma  laboral  trabajadores  empresa  hernando  rumores  despido  despidos  ley  flexibilidad
Cluster 5:  eta  civil  ley  terroristas  guardia  terrorista  víctimas  disolución  derecho  nacional
Cluster 6:  cataluña  generalitat  españa  catalanes  financiación  constitución  déficit  ciudadanos  público  económica
Cluster 7:  mujeres  violencia  género  igualdad  mujer  víctimas  ley  rumores  hombres  menores
Cluster 8:  medidas  bankia  banco  decreto  tema  ley  real  comisión  millones  vivienda
Cluster 9

## El mateix amb keywords

In [46]:
'''
vectorizer = TfidfVectorizer(vocabulary = set_docs_keywords, max_df=0.95, min_df=2,ngram_range=(1,1)) 
X = vectorizer.fit_transform(text_documents)
'''
set_docs_tokenized_keywords_without = remove_custom_set_of_words(set_docs_tokenized_keywords,stopwords)

vectorizer = TfidfVectorizer(vocabulary = set_docs_tokenized_keywords_without, max_df=0.95, min_df=2,ngram_range=(1,1)) 
X = vectorizer.fit_transform(text_documents)

In [47]:
true_k = 20
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1, verbose=True)

In [48]:
print "Clustering sparse data with %s" % km
t0 = time()
km.fit(X)
print "done in %0.3fs" % (time() - t0)

Clustering sparse data with KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=20, n_init=1,
    n_jobs=1, precompute_distances=True, random_state=None, tol=0.0001,
    verbose=True)
Initialization complete
Iteration  0, inertia 2105.426
Iteration  1, inertia 1129.159
Iteration  2, inertia 1123.435
Iteration  3, inertia 1119.648
Iteration  4, inertia 1116.996
Iteration  5, inertia 1115.343
Iteration  6, inertia 1113.490
Iteration  7, inertia 1112.191
Iteration  8, inertia 1111.548
Iteration  9, inertia 1111.261
Iteration 10, inertia 1111.062
Iteration 11, inertia 1111.037
Iteration 12, inertia 1111.001
Iteration 13, inertia 1110.980
Converged at iteration 13
done in 0.280s


In [49]:
def get_docs_in_cluster(cluster_index):
    docs_in_cluster = []
    for doc_index, doc_in_cluster_index in enumerate(km.labels_):
        if doc_in_cluster_index == cluster_index:
            docs_in_cluster.append(doc_index)
    return docs_in_cluster

print "Top 10 terms per cluster:"
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
total_docs_in_clusters = 0
for i in range(true_k):
    docs_in_cluster = get_docs_in_cluster(i)
    print "Cluster %d:" % i, len(order_centroids), ' '.join(' %s' % terms[ind] for ind in order_centroids[i, :10])
    print '\tnum. docs:', len(docs_in_cluster)
    total_docs_in_clusters += len(docs_in_cluster)
print "total_docs_in_clusters:", total_docs_in_clusters
print "total docs", len(text_documents)

doc_in_cluster_list = []
for doc_index, doc_in_cluster_index in enumerate(km.labels_):
    doc_in_cluster_list.append((doc_index, 
                                documents[doc_index]['date'], 
                                doc_in_cluster_index, 
                                documents[doc_index]['question'],
                                documents[doc_index]['keywords'],
                               ))

Top 10 terms per cluster:
Cluster 0: 20  andalucía  jóvenes  junta  oportunidades  andaluces  juvenil  formación  paro  estrategia  social
	num. docs: 57
Cluster 1: 20  becas  estudiantes  sistema  universidad  educativo  curso  erasmus  ayudas  abandono  beca
	num. docs: 42
Cluster 2: 20  sanidad  salud  sanitaria  autónomas  comunidades  pública  sanitarios  sanitario  medicamentos  ciudadanos
	num. docs: 55
Cluster 3: 20  seguridad  fiscal  inmigración  justicia  interior  fiscalía  general  caso  indulto  decisión
	num. docs: 37
Cluster 4: 20  sector  actividad  industria  industrial  economía  política  empresas  plan  turismo  cultural
	num. docs: 54
Cluster 5: 20  economía  banco  datos  bankia  crecimiento  europa  tema  vivienda  preferentes  española
	num. docs: 111
Cluster 6: 20  tributaria  agencia  fraude  fiscal  recaudación  lucha  contribuyentes  amnistía  hacienda  impuestos
	num. docs: 46
Cluster 7: 20  canarias  prospecciones  petróleo  canario  autónoma  incendios  

In [54]:
import pandas as pd
df = pd.DataFrame(doc_in_cluster_list,columns=['doc_id', 'date', 'cluster', 'question','keywords'])

# Carga de 'month' y 'year'

s = df['date']
tm = s[:]
print len(tm)
 
df['month'] = [t.month for t in tm]
df['year']  = [t.year for t in tm]

1221


In [55]:
df

,doc_id,date,cluster,question,keywords,month,year
0,0,2012-02-08,9,QUÉ REFORMA LABORAL ESTÁ PREPARANDO EL GOBIERN...,"[reforma, laboral, reforma, laboral, breve, pe...",2,2012
1,1,2012-02-08,6,CUÁNDO PIENSA CUMPLIR EL GOBIERNO PUNTUALMENTE...,"[devolución, compromisos, iva, difíciles, prof...",2,2012
2,2,2012-02-08,12,CONSIDERA QUE TIENE UN PROGRAMA DE GOBIERNO A ...,"[necesidades, altura, huelga, general, carácte...",2,2012
3,3,2012-02-08,0,CONSIDERA LA VICEPRESIDENTA QUE ESTÁ CUMPLIEND...,"[funciones, difícil, sentirme, difícil, don, a...",2,2012
4,4,2012-02-08,16,MANTIENE EL GOBIERNO SU INTENCIÓN DE IMPONER E...,"[conjunto, contrarreformas, intención, servici...",2,2012
5,5,2012-02-08,16,CREE EL GOBIERNO QUE DEBEMOS RETROCEDER VEINTI...,"[salud, sexual, interrupción, voluntaria, repr...",2,2012
6,6,2012-02-08,14,QUÉ VALORACIÓN HACE EL GOBIERNO DE LA SITUACIÓ...,"[transporte, aéreo, valoración, situación, tra...",2,2012
7,7,2012-02-15,10,QUÉ INICIATIVAS EMPRENDERÁ SU GOBIERNO PARA ME...,"[iniciativas, emprenderá, calidad, institucion...",2,2012
8,8,2012-02-15,14,QUÉ MEDIDAS ESTÁ ADOPTANDO EL GOBIERNO PARA LA...,"[inmediata, implantación, arrendamiento, finan...",2,2012
9,9,2012-02-15,9,CONSIDERA EL PRESIDENTE QUE LAS MEDIDAS QUE ES...,"[medidas, pérez, rubalcaba, principal, objetiv...",2,2012


#### Save csv

In [56]:
df.to_csv('df_doc_cluster_congres.csv', encoding='utf-8')

#### Save classifier

In [57]:
import pickle

# Save
with open('km_20150419.pkl', 'wb') as output:
    pickle.dump(km, output, pickle.HIGHEST_PROTOCOL)

# Load
with open('km_20150419.pkl', 'rb') as input:
    km_loaded = pickle.load(input)


In [58]:
def get_docs_in_cluster(cluster_index):
    docs_in_cluster = []
    for doc_index, doc_in_cluster_index in enumerate(km_loaded.labels_):
        if doc_in_cluster_index == cluster_index:
            docs_in_cluster.append(doc_index)
    return docs_in_cluster

print "Top 10 terms per cluster:"
order_centroids = km_loaded.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
total_docs_in_clusters = 0
for i in range(true_k):
    docs_in_cluster = get_docs_in_cluster(i)
    print "Cluster %d:" % i, len(order_centroids), ' '.join(' %s' % terms[ind] for ind in order_centroids[i, :10])
    print '\tnum. docs:', len(docs_in_cluster)
    total_docs_in_clusters += len(docs_in_cluster)
print "total_docs_in_clusters:", total_docs_in_clusters
print "total docs", len(text_documents)


Top 10 terms per cluster:
Cluster 0: 20  andalucía  jóvenes  junta  oportunidades  andaluces  juvenil  formación  paro  estrategia  social
	num. docs: 57
Cluster 1: 20  becas  estudiantes  sistema  universidad  educativo  curso  erasmus  ayudas  abandono  beca
	num. docs: 42
Cluster 2: 20  sanidad  salud  sanitaria  autónomas  comunidades  pública  sanitarios  sanitario  medicamentos  ciudadanos
	num. docs: 55
Cluster 3: 20  seguridad  fiscal  inmigración  justicia  interior  fiscalía  general  caso  indulto  decisión
	num. docs: 37
Cluster 4: 20  sector  actividad  industria  industrial  economía  política  empresas  plan  turismo  cultural
	num. docs: 54
Cluster 5: 20  economía  banco  datos  bankia  crecimiento  europa  tema  vivienda  preferentes  española
	num. docs: 111
Cluster 6: 20  tributaria  agencia  fraude  fiscal  recaudación  lucha  contribuyentes  amnistía  hacienda  impuestos
	num. docs: 46
Cluster 7: 20  canarias  prospecciones  petróleo  canario  autónoma  incendios  

In [27]:

new_docs = [u"la pobreza infantil es un asunto muy delicado dentro de la reforma", 
            u"la corrupción afecta a todos los partidos políticos", 
            u"la reforma laboral no ha ido bien",
            u"los precios de las tarífas eléctricas han subido mucho este último año"]
mat = vectorizer.transform(new_docs)
# print vectorizer.vocabulary_
print vectorizer.inverse_transform(mat)

[array([u'asunto', u'infantil', u'reforma', u'pobreza', u'delicado'], 
      dtype='<U20'), array([u'corrupci\xf3n', u'afecta', u'pol\xedticos', u'partidos'], 
      dtype='<U20'), array([u'laboral', u'reforma'], 
      dtype='<U20'), array([u'precios', u'el\xe9ctricas', u'a\xf1o'], 
      dtype='<U20')]


In [35]:
import pymongo
import nltk
import datetime
import re

doc_col = db['document']

doc_start_date = "08/02/2012"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
doc_end_date = "11/03/2015"
d_doc_end_date = datetime.datetime.strptime(doc_end_date, "%d/%m/%Y")

result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)

documents = [] # documents = [d1=p1+i11+...+i1M, ..., dN=pN+iN1+...]
dframe = []
jj = 0

if result.count() > 0 :
    for doc in result:
        #print doc['description']
        lq = 0
        for ii, dialogo in enumerate(doc['session_dictionary']):
            jj = jj + 1
            #print "Documento # %d" %jj
            #print "Question %s" %dialogo['question']
            #print "Num keywords pregunta: %d " % len(dialogo['keywords'])
            texto = ' '.join(p.findall(dialogo['question'])) + ' '
            li = 0
            lik = 0
            for intervencion in dialogo['intervention_dictionary']:
                texto = texto + ' ' + ' '.join(p.findall(intervencion['text']))
                #print "Num keywords intervención: %d " % len(intervencion['keywords'])
                try:
                    lik = lik + len(intervencion['keywords'])
                except:
                    lik = lik + 0
                if intervencion['text'] == '':
                    li = 0
                #print jj
                #print '*' + dialogo['question'] + "*"
                #print doc['date']
                #print dialogo['num_exp']
                else:
                    li = len(intervencion['text'].split(' '))
                    
            documents.append(texto)
            
            try:
                lqk = len(dialogo['keywords'])
            except:
                lqk = 0
            
            if dialogo['question'] == '':
                lq = 0
                #print jj
                #print '*' + dialogo['question'] + "*"
                #print doc['date']
                #print dialogo['num_exp']
            else:
                lq = len(dialogo['question'].split(' '))
                
            
                
            dframe.append((jj, dialogo['question'], (lq + li), (lqk + lik) , doc['date'], dialogo['num_exp'] ))

import pandas as pd
df = pd.DataFrame(dframe,columns=['num_doc', 'question', 'ldoc', 'lkws', 'fecha', 'num_exp'])
df.set_index('num_doc')

,question,ldoc,lkws,fecha,num_exp
num_doc,,,,,
1,QUÉ REFORMA LABORAL ESTÁ PREPARANDO EL GOBIERN...,405,24,2012-02-08,180/000001
2,"CUÁNDO PIENSA CUMPLIR EL GOBIERNO, PUNTUALMENT...",311,46,2012-02-08,180/000004
3,CONSIDERA QUE\n TIENE UN PROGRAMA DE GOBIER...,229,11,2012-02-08,180/000007
4,CONSIDERA LA VICEPRESIDENTA QUE ESTÁ\n CUMP...,519,33,2012-02-08,180/000008
5,MANTIENE EL\n GOBIERNO SU INTENCIÓN DE IMPO...,542,33,2012-02-08,180/000009
6,CREE EL GOBIERNO\n QUE DEBEMOS RETROCEDER V...,527,37,2012-02-08,180/000016
7,QUÉ VALORACIÓN\n HACE EL GOBIERNO DE LA SIT...,431,32,2012-02-08,180/000012
8,QUÉ INICIATIVAS EMPRENDERÁ SU GOBIERNO PARA ME...,350,56,2012-02-15,180/000017
9,QUÉ\n MEDIDAS ESTÁ ADOPTANDO EL GOBIERNO PA...,118,59,2012-02-15,180/000019
